In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from datetime import datetime
import base64
import hashlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Caminho para o arquivo xlsx local
file_path = './banco_dados.xlsx'

# Leitura do arquivo xlsx usando pandas
df = pd.read_excel(file_path)

In [ ]:
# Funções de conversão
def string_to_unique_float2(s: str) -> float:
    # Gera um hash MD5 da string
    md5_hash = hashlib.md5(s.encode()).hexdigest()
    # Converte o hash em um número inteiro
    int_value = int(md5_hash, 16)
    # Normaliza o valor para o intervalo entre 0 e 1
    normalized_value = int_value % (10**8) / (10**8)
    return normalized_value

def string_to_unique_float(texto: str) -> float:
    texto_bytes = texto.encode('utf-8')
    numero_base64 = base64.b64encode(texto_bytes)
    numero_unico = int.from_bytes(numero_base64, byteorder='big')
    max_valor = 2 ** (len(numero_base64) * 8) - 1  # Maior valor possível para o número
    return numero_unico / max_valor

def string_para_numero_unico(texto):
    hash_md5 = hashlib.md5(texto.encode())
    numero_unico = int(hash_md5.hexdigest(), 16)
    return numero_unico

def string_para_numero(texto):
    texto_bytes = texto.encode('utf-8')
    numero_base64 = base64.b64encode(texto_bytes)
    numero_unico = int.from_bytes(numero_base64, byteorder='big')
    return numero_unico

def numero_para_string(numero):
    numero_bytes = numero.to_bytes((numero.bit_length() + 7) // 8, byteorder='big')
    texto_original = base64.b64decode(numero_bytes).decode('utf-8')
    return texto_original

# Função para extrair data e valores, substituindo valores por números
def extrair_data_valores(data, linha):
    value = []
    timeElipsed = []
    time_point_anterior = 0
    for j, colStr in enumerate(data.columns):
        if colStr.startswith("Tela") and data[colStr].iloc[0] != "":
            colNum = int(re.search(r"Tela (\d+)", colStr).group(1))
            
            # Converte `data.iloc[linha, j]` para string
            cell_value = str(data.iloc[linha, j])
            
            match = re.match(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) - (.+)", cell_value)
            if match:
                time_point_atual = pd.to_datetime(match.group(1))
                if colNum == 1:
                    time_point_anterior = time_point_atual
                else:
                    if colNum != 42 and colNum != 74 and colNum != 75:
                        valores = match.group(2)
                        valores = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', valores)
                        valores = valores.replace("other;", "").replace("Sucess;", "").replace("Sucess", "")
                        valueAux = list(map(string_para_numero_unico, valores.split(";")))
                        timeElipsed += [(time_point_atual - time_point_anterior).total_seconds() for _ in range(len(valueAux))]
                        time_point_anterior = time_point_atual
                        value += valueAux
    return value, timeElipsed

In [ ]:
# Loop para processar colunas que começam com "Tela"
valueAux, timeElipsedAux = extrair_data_valores(df, 0)
value = np.matrix(valueAux)
timeElipsed =np.matrix(timeElipsedAux)
for linha in range(df.shape[0]-1):
    valueAux, timeElipsedAux = extrair_data_valores(df, linha+1)
    value = np.vstack(value, np.array(valueAux))
    timeElipsed =np.vstack(timeElipsed, np.array(timeElipsedAux))

# Exibir os resultados para conferência
# print(numero_para_string(value[15][13]))

# Conversão das listas para matrizes numpy
value = np.array(value, dtype=object)
timeElipsed = np.array(timeElipsed, dtype=object)

# Certifique-se de que todos os arrays em `value` e `timeElipsed` tenham o mesmo comprimento
# max_len = max(len(row) for row in value)
# value = np.array([np.pad(row, (0, max_len - len(row)), constant_values=np.nan) for row in value], dtype=float)
# timeElipsed = np.array([np.pad(row, (0, max_len - len(row)), constant_values=np.nan) for row in timeElipsed], dtype=float)

# Remove linhas com NaN no target (coluna 13)
# mask = ~np.isnan(value[:, 13])
# value = value[mask]
# timeElipsed = timeElipsed[mask]

# Separando a coluna 13 como target e criando a matriz de features
X = np.hstack([value, timeElipsed])  # Adiciona `timeElipsed` como uma feature extra
y = value[:, 13]  # Coluna 13 é o target

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalonamento dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir a estrutura da rede neural com TensorFlow
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Saída com um único valor para regressão
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=5000, batch_size=32, validation_split=0.2, verbose=1)

# Avaliar o modelo no conjunto de teste
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f'MAE no conjunto de teste: {test_mae}')
